<a href="https://colab.research.google.com/github/muhiqbalalamin/DeepLearning-UAS/blob/main/Chapter_13_Teori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading and Preprocessing Data with TensorFlow**

## Tujuan Bab
Bab ini membahas cara:

* Memuat data secara efisien menggunakan tf.data

* Melakukan prapemrosesan data secara modular dan skalabel

* Mengoptimalkan pipeline input untuk performa maksimal

## Tantangan dalam Pipeline Data Input
1. Dataset berukuran besar tidak dapat dimuat seluruhnya ke dalam memori.

2. Dibutuhkan proses prapemrosesan yang cepat dan konsisten.

3. Pelatihan model yang efisien memerlukan pipeline input yang dioptimalkan.

## API tf.data
TensorFlow menyediakan API tf.data untuk:

* Membuat objek Dataset

* Melakukan transformasi data

* Menangani pipeline streaming secara efisien (lazy evaluation)

## Pembuatan Dataset
Beberapa cara umum:

* Dari array: from_tensor_slices()

* Dari file: TextLineDataset, TFRecordDataset

* Dari generator Python

## Transformasi Dataset
Objek Dataset bersifat lazy, artinya transformasi dilakukan hanya saat dibutuhkan.

Transformasi umum meliputi:

* map(): menerapkan transformasi pada tiap elemen

* filter(): menyaring elemen berdasarkan kondisi

* batch(): mengelompokkan data ke dalam batch

* shuffle(): mengacak urutan data

* repeat(): mengulang dataset

## Optimalisasi Performa Pipeline
### Prefetching
prefetch(n) memungkinkan proses pemuatan data dan pelatihan berjalan secara paralel.

### Interleaving & Parallel Mapping
* interleave(): membaca banyak file secara bersamaan

* map(..., num_parallel_calls=tf.data.AUTOTUNE): memproses data secara paralel

## Caching
cache() menyimpan hasil transformasi agar tidak dihitung ulang di setiap epoch.

### Format Data: TFRecord
TFRecord adalah format biner yang efisien untuk menyimpan data dalam skala besar:

* Kompresi tinggi

* Akses cepat

* Cocok untuk produksi (digunakan oleh Google dan TFX)

Digunakan bersama dengan:

* TFRecordDataset

* tf.train.Example (untuk parsing protokol buffer)

## Parsing dan Decoding TFRecord
Setelah data dibaca dari file TFRecord, perlu di-decode agar dapat digunakan kembali sesuai struktur aslinya.

### Contoh decoding:
* Gambar:

    * JPEG → tf.io.decode_jpeg()

    * PNG → tf.io.decode_png()

* Teks atau bytes:

    * tf.io.decode_raw()

* Label numerik:

    * Gunakan tf.cast() ke tf.int64 atau tf.float32

* Parsing TFRecord:

feature_description = {
  'image': tf.io.FixedLenFeature([], tf.string),
  'label': tf.io.FixedLenFeature([], tf.int64)
}

parsed_example = tf.io.parse_single_example(serialized_example, feature_description)

## Prapemrosesan Data (Preprocessing)
Transformasi umum sebelum data masuk ke model:

* Normalisasi: skala nilai ke rentang 0–1

* Standarisasi: mengurangkan rata-rata dan membagi dengan standar deviasi

* Resizing: menyesuaikan ukuran gambar (misalnya menjadi 224x224 piksel)

* Augmentasi: rotasi, flipping, cropping, zoom untuk memperkuat generalisasi model

* Tokenisasi: memecah teks menjadi unit kata/karakter

* Padding: menyamakan panjang input teks dalam batch

### Lokasi Preprocessing
1. Offline
Dilakukan sebelum pelatihan dan disimpan ke disk (dalam format .npy, TFRecord, dll).

2. Di dalam pipeline tf.data
Menggunakan fungsi map() untuk menerapkan transformasi secara on-the-fly.

3. Di dalam model
Menggunakan layer seperti Rescaling, Resizing, TextVectorization.

## Perbandingan Lokasi Preprocessing
| Lokasi             | Keunggulan                              | Kelemahan                             |
| ------------------ | --------------------------------------- | ------------------------------------- |
| **Di dalam model** | Portabel, bisa dibawa saat *deployment* | Dapat memperlambat proses inferensi   |
| **Dalam pipeline** | Cepat, fleksibel selama pelatihan       | Tidak otomatis ikut saat ekspor model |
| **Offline**        | Konsisten, mudah didistribusikan        | Tidak fleksibel jika data berubah     |


## Optimalisasi Pipeline
Untuk efisiensi maksimal, urutan transformasi yang disarankan adalah:

Load → Shuffle → Cache → Batch → Prefetch

## TensorFlow Extended (TFX)
TFX adalah platform produksi untuk machine learning berbasis TensorFlow, yang mendukung pipeline skala besar dari awal hingga deployment.

## Komponen Utama TFX:
* ExampleGen: mengimpor dan mengonversi data ke format TFRecord

* StatisticsGen: menghasilkan statistik dataset

* SchemaGen: membuat skema fitur berdasarkan statistik

* ExampleValidator: mendeteksi data yang rusak, hilang, atau anomali

* Transform: melakukan prapemrosesan berskala besar dengan TensorFlow Transform (TFT)

* Trainer: melatih model dengan Keras atau Estimator

* Tuner (opsional): optimasi otomatis hyperparameter

* Evaluator: mengevaluasi performa model

* InfraValidator: memastikan model dapat disajikan sebelum deployment

* Pusher: men-deploy model ke TensorFlow Serving atau sistem lainnya

## Rumus dan Konsep Kunci
### Normalisasi

\[
$x_{\text{norm}} = \frac{x - \mu}{\sigma}$
\]

- \($ x $\): nilai asli
- \($ \mu $\): rata-rata
- \($ \sigma $\): deviasi standar

### Ukuran Buffer Shuffle
* Buffer yang lebih besar → pengacakan data lebih merata

* Ideal jika buffer_size >= jumlah data, selama memori mencukupi

## Pipeline tf.data yang Dioptimalkan
Gunakan transformasi berikut dalam urutan yang direkomendasikan:

dataset = dataset.shuffle(buffer_size)
                 .cache()
                 .batch(batch_size)
                 .prefetch(tf.data.AUTOTUNE)

* shuffle(): mencegah model menghafal urutan data

* cache(): menyimpan hasil transformasi agar tidak diulang

* batch(): mempercepat pelatihan dengan proses paralel

*prefetch(): memperlancar alur antara CPU dan GPU

# **Kesimpulan**
* API tf.data menawarkan pipeline data yang efisien dan dapat diskalakan, cocok untuk berbagai skenario — dari eksperimen lokal hingga pelatihan skala besar.

* Format TFRecord sangat cocok untuk penyimpanan data dan pelatihan produksi.

* Prapemrosesan dapat dilakukan secara offline, di pipeline tf.data, atau langsung di dalam model — masing-masing memiliki kelebihan dan kekurangan.

* Gunakan TFX untuk membangun pipeline machine learning end-to-end secara terstandar dan andal.

* Optimalisasi pipeline input sangat penting untuk efisiensi waktu pelatihan dan pemanfaatan sumber daya secara maksimal.

# **Referensi**
Géron, A. (2019). Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow. O'Reilly Media.